# 制作数据集

In [ ]:
import os
import numpy as np
videodir = 'D:/py/My_work/video/'
dirlist = os.listdir(videodir)
category = np.arange(6).tolist()
C = [[] for i in range(6)]
for i in dirlist:
    strtemp = i.split('-')[0]
    M = int(strtemp) if strtemp.isdigit() else strtemp
    if M in category:
        C[category.index(M)].append(i)

In [ ]:
# 采集右半屏区域 去除背景后的手势
import cv2

def cutimg(imgpath,dirname,videoname,tot = 0):
    # 开启电脑上摄像头进行实时预测
    cap = cv2.VideoCapture(imgpath)
    # 循环使用cv2的read()方法读取视频帧
    while 1:
        ret, img = cap.read()
        if ret:
            filename = dirname +'/'+videoname.split('.')[0]+'_'+ str(tot) + '.jpg'
            filename1 = dirname +'/'+videoname.split('.')[0]+'_flip_'+ str(tot) + '.jpg'
          # img = bs.apply(frame)
            cap_region_x_begin = 0.5  # start point/total width
            cap_region_y_end = 1  # start point/total width

            img = img[0:int(cap_region_y_end * img.shape[0]),
                  0:int(cap_region_x_begin * img.shape[1])]  # clip the ROI
            img1 = cv2.flip(img, 1)  # flip the frame horizontally
            if  tot<50:
                filename = dir+'-1/'+videoname.split('.')[0]+'_'+ str(tot) + '.jpg'
                filename1 =  dir+'-1/'+videoname.split('.')[0]+'_flip_'+ str(tot) + '.jpg'
            cv2.imwrite(filename, img)
            cv2.imwrite(filename1, img1)
                # cv2.imshow('frame2', img)
            if tot%50 ==0:
                print('tot=',tot, filename)
            # break
            tot += 1
        
            k = cv2.waitKey(1) & 0xff
            if k == 27:
                break
        else:
            break
       
    cap.release()
    cv2.destroyAllWindows()
#     return tot

In [ ]:
dir = 'dataset2/'
for i in C:
    count = 0
    cla = i[0].split('-')[0]
    # print(i)
    dirname = dir+cla
    os.makedirs(dirname,exist_ok = True)
    for j in i:
        # print(videodir+j)
        # imgpath = 'D:/py/My_work/video/3-2.mp4'   
        print("进入视频{}采集".format(j))
        cutimg(videodir+j,dirname,j)
#         print("count now is ",count)
    print("类别{}采集完毕".format(cla))
    # strtemp = i.split('-')[0]

# 训练模型

In [ ]:
from fastai.vision import *
np.random.seed(42)
path = pathlib.Path('dataset2')
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.4,
        ds_tfms=get_transforms(), size=128, num_workers=0).normalize(imagenet_stats)# size的设置不超过224，不然内存不够
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
# 查看分类 、训练集 、 验证集的数量
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
# 然后我们运行学习率查找方法，把它的结果画出来（它会告诉你输入什么代码来画出结果）。我们看下这个图。
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-3,1e-1))

In [ ]:
# 我喜欢把开始的结果保存下来，res34 size224
name = '../../model/10-times-res34-0137'
learn.save(name)

In [ ]:
#像以前一样，我们可以使用ClassificationInterpretation类型来看看结果。
learn.load(name);

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
# 首先将模型导出：
#这会在目录中创建一个名为export.pkl的文件，它包含了部署模型所需要的所有内容（模型，权重以及一些元数据）
learn.export()

In [ ]:
#export.pkl，然后我们在生产环境中创建学习器：

learn = load_learner(path)

# 对已命名的数字文件夹进行预测，并显示准确率

In [ ]:
import cv2
from fastai.vision import *
from collections import Counter
path = pathlib.Path('dataset2')
def backgroudRemove(imgpath):
    # 加载本地视频进行预测
    cap = cv2.VideoCapture(imgpath)
    # 开启电脑上摄像头进行实时预测
#     cap = cv2.VideoCapture(0)
    bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)
    tot = 1
    # 读取训练的模型进行预测
    learn = load_learner(path)
    #设置起始预测值为 -1
    preNum = -1
    count = 0
    num = (imgpath.split('/')[-1]).split('-')[0]
    num = num if num.isdigit() else -1
    Res = [-1 for i in range(8)]
    # 循环使用cv2的read()方法读取视频帧
    while 1:
        ret, img = cap.read()
        if ret:
         
#             frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
          
#             img = bs.apply(frame)
            
      
            cap_region_x_begin = 0.5  # start point/total width
            cap_region_y_end = 1  # start point/total width

            img = img[0:int(cap_region_y_end * img.shape[0]),
                  0:int(cap_region_x_begin * img.shape[1])]  # clip the ROI
            img = cv2.flip(img, 1)  # flip the frame horizontally
#             cv2.imshow('frame1', img)
#             cv2.imwrite('temp.jpg', img)

            img_fastai = Image(pil2tensor(img, dtype=np.float32).div_(255)) #已进行归一化处理
            pred_class, pred_idx, outputs = learn.predict( img_fastai)
            
            # 将预测结果保存下来进行查看或者作为数据集添加数据
            # cv2.imwrite('D:/py/10.0-2.13-opencv/jupyter-notebook/work1_hand_reg/img_cut/' +str(a)+'_'+ str(tot) + '.jpg', img)
            
            Res.pop(0)
            Res.append(int(str(pred_class)))
            res = Counter(Res).most_common(1)[0][0]
            cv2.putText(img,str(res), (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 10, cv2.LINE_AA)
#             cv2.putText(img,str(pred_class), (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 10, cv2.LINE_AA)
            cv2.imshow('frame2', img)
            
#             res = 0
#             if int(res)!=int(num) and tot>72:
#                 count +=1
#                 if  count%10==0:
#                     print("预测错误，序号为：{}，真实值为 {}, 预测值为：{},预测错误累计次数为：{}".format(tot-72,num,preNum,count))
    
            k = cv2.waitKey(1) & 0xff
            if k == 27:
                break
        else:
            break
        tot += 1
#     print("错误率为{:.4f} %,预测数量为 {}，错误数量为：{}".format(count/(tot-72)*100,tot-72,count))
    
    cap.release()
    cv2.destroyAllWindows()
    return imgpath+"错误率为 ： "+str(count/(tot-50))

dir = 'D:/py/My_work/video/'
# dirlist = os.listdir(dir)
dirlist = 'demo2.mp4'
res = []
backgroudRemove(dir + dirlist)
# for videopath in dirlist:
#     print(dir + videopath)
#     r = backgroudRemove(dir + videopath)
#     res.append(r)

#3-1  3-2  4-1 4-2 5-1 错误率高于20%    采用24个连续的预测均值后错误率下降到10%

# 用百度API进行预测，速度太慢，达不到实时的要求

In [ ]:
# encoding:utf-8
import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=Q1rsKyaEdy4T6gW0MpsAGza3&client_secret=NuXBpF0W5XTvX4aNO0E4juZfacFG5qwX'
response = requests.get(host)
if response:
    print(response.json()[])

In [ ]:
import requests
import base64

def filename(imgpath):
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/gesture"
    # 二进制方式打开图片文件
    f = open(imgpath, 'rb')
    img = base64.b64encode(f.read())

    params = {"image":img}
    access_token = '24.6bd8a1f2fbe078f228c130868f8c2f5f.2592000.1591627146.282335-19801644'
    request_url = request_url + "?access_token=" + access_token
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    response = requests.post(request_url, data=params, headers=headers)
    if response:
        classname = response.json()['result'][0]['classname']
        probability = response.json()['result'][0]['probability']
        print (classname,probability)
        return classname,probability

In [ ]:
filename('imgforBaidu/293.jpg')

# 断点续训

In [ ]:
# Checkpoint the weights when validation accuracy improves
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy
# fix random seed for reproducibility
seed= 7
numpy.random.seed(seed)
# load pima indians dataset
dataset= numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X= dataset[:,0:8]
Y= dataset[:,8]
# create model
model= Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
weightpath = "./checkpoint/weights-best.hdf5"
if os.path.exists(weightpath):
    model.load_weights(filepath)
    # 若成功加载前面保存的参数，输出下列信息
    print("checkpoint_loaded")
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
filepath="./checkpoint/weights-best.hdf5"
checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list= [checkpoint]
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=15, batch_size=10, callbacks=callbacks_list, verbose=0)